In [1]:
#!pip install PhiFlow
!pip install git+https://github.com/tum-pbs/PhiFlow.git@2.3.0

  Cloning https://github.com/tum-pbs/PhiFlow.git (to revision 2.3.0) to /tmp/pip-req-build-gihk80w_
  Running command git clone --filter=blob:none --quiet https://github.com/tum-pbs/PhiFlow.git /tmp/pip-req-build-gihk80w_
  Running command git checkout -q f8d0009b50682e2b77c5decd5c0aa7e3a473cdd7
  Resolved https://github.com/tum-pbs/PhiFlow.git to commit f8d0009b50682e2b77c5decd5c0aa7e3a473cdd7
  Preparing metadata (setup.py) ... done


In [3]:
import numpy as np
from phi.flow import *
import torch
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import matplotlib.animation as animation

## Translation: Two Buoyant forces flow

In [ ]:
try:
    os.mkdir("Translation")
except:
    print("Translation folder already exists")


inflow_lst = [(i, j) for i in np.arange(8,64,8) for j in np.arange(5,30,5)]

res_x = 64
res_y = 64
pressure = None
DOMAIN = dict(x=res_x, y=res_y)
DOMAIN_LEFT = dict(x=res_x, y=res_y)
DOMAIN_RIGHT = dict(x=res_x, y=res_y)

LEFT = StaggeredGrid(HardGeometryMask(Box['x,y', :res_x//2, :]), extrapolation.ZERO, **DOMAIN_LEFT)
RIGHT = StaggeredGrid(HardGeometryMask(Box['x,y', res_x//2:, :]), extrapolation.ZERO, **DOMAIN_RIGHT)

smoke = CenteredGrid(0, extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])  # sampled at cell centers
velocity = StaggeredGrid(0, extrapolation.ZERO, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])# sampled in staggered form at face centers

INFLOW_LOCATION = tensor(inflow_lst, batch('inflow_loc'), channel(vector='x,y'))
#INFLOW_LOCATION.name = 'name'
print(INFLOW_LOCATION)

INFLOW = CenteredGrid(Sphere(center = INFLOW_LOCATION, radius=5), extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y',0:res_x, 0:res_y])

data = []
for _ in view(smoke, velocity, description = 'hello very cool', gui='console').range(310):
    smoke = advect.mac_cormack(smoke, velocity, dt=1) + INFLOW
    buoyancy_force1 = smoke * LEFT * (0, 0.1) @ velocity#(velocity * LEFT)
    buoyancy_force2 = smoke * RIGHT * (0, 0.2) @ velocity
    velocity = advect.semi_lagrangian(velocity, velocity, dt=0.5) + buoyancy_force1 + buoyancy_force2
    velocity, _ = fluid.make_incompressible(velocity, (), Solve('auto', 1e-5, 0, x0=pressure))
    data.append(torch.from_numpy(velocity.staggered_tensor().numpy(['inflow_loc', 'vector', 'y', 'x'])).float().unsqueeze(1))

data = torch.cat(data, dim = 1)[:,:,:,:-1,:-1]
for k in range(len(inflow_lst)):
    torch.save(data[k].double().float(), "Translation/raw_data_" + str(inflow_lst[k][0]) + "_" + str(inflow_lst[k][1]) + ".pt")

<ipython-input-3-c350699a87ab>:16: DeprecationWarning: HardGeometryMask and SoftGeometryMask are deprecated. Use field.mask or field.resample instead.
  LEFT = StaggeredGrid(HardGeometryMask(Box['x,y', :res_x//2, :]), extrapolation.ZERO, **DOMAIN_LEFT)
<ipython-input-3-c350699a87ab>:17: DeprecationWarning: HardGeometryMask and SoftGeometryMask are deprecated. Use field.mask or field.resample instead.
  RIGHT = StaggeredGrid(HardGeometryMask(Box['x,y', res_x//2:, :]), extrapolation.ZERO, **DOMAIN_RIGHT)


(inflow_locᵇ=35, vectorᶜ=x,y) int64 23.500 ± 15.008 (5e+00...6e+01)
<cell line: 29>
hello very cool

PhiFlow console interface active. Type 'help' for a list of available commands.
PhiFlow>

KeyboardInterrupt: 

## Rotation

In [ ]:
try:
    os.mkdir("Rotation")
except:
    print("Translation folder already exists")

inflow_pos = [[(32,5), (0,0.01)], [(64-5,32), (-0.01, 0)], [(32,64-5),(0, -0.011)], [(5,32), (0.01, 0)]]

k = 0
for i in np.arange(9):
    for j in range(4):
        k += 1
        res_x = 64
        res_y = 64
        pressure = None
        DOMAIN = dict(x=res_x, y=res_y, bounds=Box['x,y',0:res_x, 0:res_y])
        smoke = CenteredGrid(0, extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])  # sampled at cell centers
        velocity = StaggeredGrid(0, extrapolation.ZERO, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])  # sampled in staggered form at face centers
        INFLOW_LOCATION = tensor([inflow_pos[j][0]], batch('inflow_loc'), channel(vector='x,y'))
        INFLOW = CenteredGrid(Sphere(center=INFLOW_LOCATION, radius=5), extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y',0:res_x, 0:res_y])

        data = []
        for _ in view(smoke, velocity, description = 'hello very cool', gui='console').range(310):
            smoke = advect.mac_cormack(smoke, velocity, dt=1) + INFLOW
            buoyancy_force = smoke * tuple(e*k for e in inflow_pos[j][1]) @ velocity
            velocity = advect.semi_lagrangian(velocity, velocity, dt=1) + buoyancy_force
            velocity, _ = fluid.make_incompressible(velocity, (), Solve('auto', 1e-5, 0, x0=pressure))
            vel_tensor = torch.from_numpy(velocity.staggered_tensor().numpy(['inflow_loc', 'vector', 'y', 'x'])).float()
            data.append(vel_tensor)
        data = torch.cat(data, dim = 0)
        torch.save(data.float()[1:,:,:64,:64], "Rotation/raw_data_"+ str(k) + "_" + str(j) + ".pt")



Translation folder already exists
<cell line: 9>
hello very cool

PhiFlow console interface active. Type 'help' for a list of available commands.
PhiFlow>

KeyboardInterrupt: 

## Scaling

In [3]:
try:
    os.mkdir("Scale")
except:
    print("Translation folder already exists")

factors = np.linspace(1, 5, 41)
for i in range(0, 38):
    factor = factors[i]
    res_x = int(64*factor)
    res_y = int(64*factor)
    pressure = None
    DOMAIN = dict(x=res_x, y=res_y, bounds=Box['x,y',0:res_x, 0:res_y])
    smoke = CenteredGrid(0, extrapolation.BOUNDARY, **DOMAIN)  # sampled at cell centers
    velocity = StaggeredGrid(0, extrapolation.ZERO, **DOMAIN)  # sampled in staggered form at face centers
    INFLOW_LOCATION = tensor([(32*factor, 5*factor)], batch('inflow_loc'), channel(vector='x,y'))
    INFLOW = CenteredGrid(Sphere(center=INFLOW_LOCATION, radius=5*factor), extrapolation.BOUNDARY, **DOMAIN)

    data = []
    for _ in view(smoke, velocity, description = 'hello very cool', gui='console').range(310):
        smoke = advect.mac_cormack(smoke, velocity, dt=1) + INFLOW
        buoyancy_force = smoke * (0, 0.1*factor) @ velocity
        velocity = advect.semi_lagrangian(velocity, velocity, dt = 0.5) + buoyancy_force
        velocity, _ = fluid.make_incompressible(velocity, (), Solve('auto', 1e-5, 0, x0=pressure))
        vel_tensor = torch.from_numpy(velocity.staggered_tensor().numpy(['inflow_loc', 'vector', 'y', 'x'])).float()
        #print(vel_tensor.shape)
        data.append(vel_tensor)

    data = torch.cat(data, dim  = 0).transpose(0,1).unsqueeze(0)
    data = F.interpolate(data, scale_factor = (1, 1/factor, 1/factor), mode="trilinear", align_corners = True).squeeze(0).transpose(0,1)/factor
    #print(data.shape)
    torch.save(data.float()[1:,:,:64,:64], "Scale/raw_data_"+ str(i) + ".pt")

Translation folder already exists
<cell line: 7>
hello very cool

PhiFlow console interface active. Type 'help' for a list of available commands.
PhiFlow>

KeyboardInterrupt: 

## Create datasets with various rotation equivariance errors

In [7]:
try:
    os.mkdir("equivariance_test")
except:
    print("Translation folder already exists")

for i in np.arange(50):
    res_x = 64
    res_y = 64
    pressure = None
    DOMAIN = dict(x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])
    smoke = CenteredGrid(0, extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])  # sampled at cell centers
    velocity = StaggeredGrid(0, extrapolation.ZERO, x=res_x, y=res_y, bounds=Box['x,y', 0:res_x, 0:res_y])  # sampled in staggered form at face centers
    INFLOW_LOCATION = tensor([(32, 5)], batch('inflow_loc'), channel(vector = 'x,y'))
    INFLOW = CenteredGrid(Sphere(center=INFLOW_LOCATION, radius=5), extrapolation.BOUNDARY, x=res_x, y=res_y, bounds=Box['x,y',0:res_x, 0:res_y])
    data = []
    for _ in view(smoke, velocity, description = 'hello very cool', gui='console').range(61):
        smoke = advect.mac_cormack(smoke, velocity, dt=1) + INFLOW
        buoyancy_force = smoke * (0, 0.005*(i+1)) @ velocity
        velocity = advect.semi_lagrangian(velocity, velocity, dt=0.5) + buoyancy_force
        velocity, _ = fluid.make_incompressible(velocity, (), Solve('auto', 1e-5, 0, x0=pressure))
        vel_tensor = torch.from_numpy(velocity.staggered_tensor().numpy(['inflow_loc', 'vector', 'y', 'x'])).float()

        data.append(vel_tensor)
    data = torch.cat(data, dim = 0)
    torch.save(data.float()[10:60,:,:64,:64], "equivariance_test/raw_data_"+ str(i) + ".pt")

Translation folder already exists
<cell line: 6>
hello very cool

PhiFlow console interface active. Type 'help' for a list of available commands.
PhiFlow>

KeyboardInterrupt: 

In [1]:
def rot_vector(inp, theta):
    #inp shape: c x 2 x 64 x 64
    theta = torch.tensor(theta).float()
    rot_matrix = torch.tensor([[torch.cos(theta), -torch.sin(theta)], [torch.sin(theta), torch.cos(theta)]]).float()
    out = torch.einsum("ab, bc... -> ac...",(rot_matrix, inp.transpose(0,1))).transpose(0,1)
    return out

def get_rot_mat(theta):
    theta = torch.tensor(theta).float()
    return torch.tensor([[torch.cos(theta), -torch.sin(theta), 0],
                         [torch.sin(theta), torch.cos(theta), 0]]).float()

def rot_img(x, theta):
    rot_mat = get_rot_mat(theta)[None, ...].float().repeat(x.shape[0],1,1)
    grid = F.affine_grid(rot_mat, x.size()).float()
    x = F.grid_sample(x, grid)
    return x.float()

def rot_field(x, theta):
    x_rot = torch.cat([rot_img(rot_vector(x, theta)[:,:1],  theta),
                       rot_img(rot_vector(x, theta)[:,-1:], theta)], dim = 1)
    return x_rot

In [4]:
direc = "equivariance_test/raw_data_"
lst = [(1,1,1,1), (1,2,3,4), (1,3,5,7), (1,4,7,10), (1,5,9,13), (1,6,11,16),
       (1,7,13,19), (1,8,15,22), (1,9,17,25), (1,10,19,28)]

EE = []
for i, item in enumerate(lst):
    d1 = torch.load(direc + str(item[0]) + ".pt")
    d2 = torch.load(direc + str(item[1]) + ".pt")
    d3 = torch.load(direc + str(item[2]) + ".pt")
    d4 = torch.load(direc + str(item[3]) + ".pt")
    torch.save(d1, "equivariance_test/E_" + str(i) + "/data_0.pt")
    torch.save(rot_field(d2, np.pi/2), "equivariance_test/E_" + str(i) + "/data_1.pt")
    torch.save(rot_field(d3, np.pi), "equivariance_test/E_" + str(i) + "/data_2.pt")
    torch.save(rot_field(d4, np.pi/2*3), "equivariance_test/E_" + str(i) + "/data_3.pt")
    # Equivariance error of each dataset
    EE.append(np.round(torch.mean((torch.abs(d1[10:-1]-d2[10:-1]) + torch.abs(d1[10:-1]-d3[10:-1]) + torch.abs(d1[10:-1]-d4[10:-1]))/3).numpy().item(),3))

FileNotFoundError: [Errno 2] No such file or directory: 'equivariance_test/raw_data_1.pt'